# Нейросетевой анализ

Мы приведём два варианта нейросетевого анализа набора данных. Первый заключается в построении полносвязной нейронной сети по изображениям, *растянутым* в вектора. 

Импортируем инструменты

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

Импортируем данные

In [2]:
data = pd.read_parquet("images.parquet")
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55322 entries, 0 to 55321
Columns: 7501 entries, red_0_0 to target
dtypes: float64(7501)
memory usage: 3.1 GB


red_0_0   red_0_1   red_0_2   red_0_3   red_0_4   red_0_5   red_0_6  \
0  0.117676  0.117676  0.117676  0.117676  0.117676  0.117676  0.117676   
1  0.846191  0.865234  0.971680  0.981934  0.987305  0.972656  0.990723   
2  0.974121  0.951172  0.700195  0.699707  0.750977  0.737793  0.982910   
3  0.723633  0.723633  0.650879  0.719727  0.747070  0.770508  0.780273   
4  0.799316  0.810547  0.820312  0.804688  0.770996  0.742188  0.724609   

    red_0_7   red_0_8   red_0_9  ...  blue_49_41  blue_49_42  blue_49_43  \
0  0.117676  0.117676  0.117676  ...    0.572754    0.554199    0.442627   
1  0.954102  0.883789  0.947266  ...    0.035309    0.003922    0.027451   
2  0.921875  0.877441  0.683105  ...    0.894043    0.851074    0.866699   
3  0.772461  0.735352  0.723633  ...    0.519043    0.513184    0.508789   
4  0.712402  0.703613  0.702148  ...    0.297607    0.471436    0.501465   

   blue_49_44  blue_49_45  blue_49_46  blue_49_47  blue_49_48  blue_49_49  \
0    0.434814    0.451172    0.470459    0.487793    0.505859    0.534180   
1    0.075989    0.111267    0.114197    0.103943    0.096069    0.075500   
2    0.886719    0.890137    0.890137    0.890137    0.890137    0.890137   
3    0.535156    0.304443    0.023300    0.033020    0.081177    0.232300   
4    0.489990    0.433350    0.377930    0.437744    0.484863    0.474854   

   target  
0     1.0  
1     1.0  
2     1.0  
3     1.0  
4     1.0  

[5 rows x 7501 columns]

Составляем матрицу объекты-признаки и вектор целевой переменной

In [3]:
X = data.drop(columns=["target"]).values
y = np.array(data["target"].values.reshape(-1, 1), dtype=np.int32)
X.shape, y.shape

((55322, 7500), (55322, 1))

Разбиваем выборку на обучающую и тестовую 

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Стандартизируем данные

In [5]:
ss = StandardScaler().fit(X_train)

X_train_scaled = ss.transform(X_train)
X_test_scaled = ss.transform(X_test)

Определяем модель

In [6]:
model = Sequential(name="full_connected")

model.add(Input(shape=(X_train_scaled.shape[1],)))
model.add(Dense(1500, name="color", dtype=np.float32)) 
model.add(Dropout(0.33))
model.add(Dense(300, activation="relu", dtype=np.float64))
model.add(Dense(60, activation="relu", dtype=np.float64))
model.add(Dense(12, activation="relu", dtype=np.float64))
model.add(Dense(1, activation="sigmoid", name="output_layer", dtype=np.float64))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

Проводим обучение

In [7]:
model.fit(X_train_scaled, y_train, epochs=4, batch_size=32)

Epoch 1/4
1159/1159 [==============================] - 97s 84ms/step - loss: 0.4240 - accuracy: 0.8406
Epoch 2/4
1159/1159 [==============================] - 87s 75ms/step - loss: 0.2946 - accuracy: 0.8809
Epoch 3/4
1159/1159 [==============================] - 88s 76ms/step - loss: 0.2860 - accuracy: 0.8830
Epoch 4/4
1159/1159 [==============================] - 86s 74ms/step - loss: 0.2641 - accuracy: 0.8926


Проверяем качество

In [8]:
y_pred = model.predict(X_test_scaled).flatten()

In [9]:
accuracy_score(y_test.flatten(), np.array(y_pred > 0.5, dtype=np.int16))

0.8837158350221833

In [10]:
roc_auc_score(y_test.flatten(), y_pred)

0.9528931790372381